# DAT 203 - Lab Assignment #2

- Author: Colin Bowers
- Date: May 13, 2023

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import sklearn as sk 

In [2]:
df = pd.read_csv("data/Titanic_missing.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          825 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         819 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
# here are the rows with missing values for Fare (edited directly in the csv)
df[ df.Fare.isnull() ]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,NaN,C23 C25 C27,S
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,NaN,NaN,Q
29,30,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,349216,NaN,NaN,S
30,31,0,1,"Uruchurtu, Don. Manuel E",male,40.0,0,0,PC 17601,NaN,NaN,C
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,PC 17569,NaN,B78,C
...,...,...,...,...,...,...,...,...,...,...,...,...
789,790,0,1,"Guggenheim, Mr. Benjamin",male,46.0,0,0,PC 17593,NaN,B82 B84,C
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,NaN,NaN,S
814,815,0,3,"Tomlin, Mr. Ernest Portage",male,30.5,0,0,364499,NaN,NaN,S
834,835,0,3,"Allum, Mr. Owen George",male,18.0,0,0,2223,NaN,NaN,S


## Question #3
Apply following data preparation rules

In [5]:
# Drop the columns where all elements are missing values:
# df.dropna(how='all’)

df2 = df.dropna(how='all', axis=1)

# NOTE: no columns dropped since none are missing all values 
print( df.columns[ df.columns.isin(df2.columns) == False ] )

Index([], dtype='object')


In [6]:
# Drop the columns where any of the elements are missing values
# df.dropna(how='any')

df3 = df.dropna(how='any', axis=1)

# The following columns were dropped: 'Sex', 'Age', 'Fare', 'Cabin', 'Embarked'
print( df.columns[ df.columns.isin(df3.columns) == False ] )

Index(['Sex', 'Age', 'Fare', 'Cabin', 'Embarked'], dtype='object')


In [7]:
# Keep only the rows which contain 2 missing values maximum
# df.dropna(thresh=2)

# threshold is 11 since 13 (columns total) - 2 (allowed to be missing)
df4 = df.dropna(thresh=11)

# 21 rows dropped
print(len(df) - len(df4))
df4

251


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [8]:
# Fill all missing values with the mean of the particular column
# df.fillna(df.mean())

m = df.mean(numeric_only=True)

df5 = df.fillna(m)

print(m)

PassengerId    446.000000
Survived         0.383838
Pclass           2.308642
Age             29.699118
SibSp            0.523008
Parch            0.381594
Fare            31.978433
dtype: float64


In [9]:
df5[ df5.Fare != df.Fare ] 

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
27,28,0,1,"Fortune, Mr. Charles Alexander",male,19.000000,3,2,19950,31.978433,C23 C25 C27,S
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,29.699118,0,0,330959,31.978433,NaN,Q
29,30,0,3,"Todoroff, Mr. Lalio",male,29.699118,0,0,349216,31.978433,NaN,S
30,31,0,1,"Uruchurtu, Don. Manuel E",male,40.000000,0,0,PC 17601,31.978433,NaN,C
31,32,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,29.699118,1,0,PC 17569,31.978433,B78,C
...,...,...,...,...,...,...,...,...,...,...,...,...
789,790,0,1,"Guggenheim, Mr. Benjamin",male,46.000000,0,0,PC 17593,31.978433,B82 B84,C
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.000000,4,2,347082,31.978433,NaN,S
814,815,0,3,"Tomlin, Mr. Ernest Portage",male,30.500000,0,0,364499,31.978433,NaN,S
834,835,0,3,"Allum, Mr. Owen George",male,18.000000,0,0,2223,31.978433,NaN,S


In [10]:
# Fill any missing value in column 'A' with the column median
# df['A'].fillna(df['A'].median())

fix_fare = df['Fare'].fillna( df["Fare"].median() )
fix_fare[ df.Fare.isna() ]

27     14.4542
28     14.4542
29     14.4542
30     14.4542
31     14.4542
        ...   
789    14.4542
813    14.4542
814    14.4542
834    14.4542
835    14.4542
Name: Fare, Length: 72, dtype: float64

In [11]:
newdf = pd.DataFrame( {
    "orig": df["Fare"], 
     "new": fix_fare
})
newdf[ newdf.orig != newdf.new ]

,orig,new
27,NaN,14.4542
28,NaN,14.4542
29,NaN,14.4542
30,NaN,14.4542
31,NaN,14.4542
...,...,...
789,NaN,14.4542
813,NaN,14.4542
814,NaN,14.4542
834,NaN,14.4542


## Question #4

Look at the statistical summary of the prepared dataset:


In [12]:
print( df.describe() )

       PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  819.000000  
mean     0.381594   31.978433  
std      0.806057   50.096958  
min      0.000000    0.000000  
25%      0.000000    7.895800  
50%      0.000000   14.454200  
75%      0.000000   30.847900  
max      6.000000  512.329200  


## Question #5
Look at the statistical summary for categorical variables:

In [13]:
categorical = df.dtypes[df.dtypes == "object"].index
print(categorical)
df[categorical].describe()

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')


,Name,Sex,Ticket,Cabin,Embarked
count,891,825,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,539,7,4,644


## Question #6
Identify categorical variables and explain why they cant be used in the raw form in the analysis

**Categorical Variables**
- Name
- Sex
- Ticket
- Cabin
- Embarked

In most cases, categorical values need to be converted into a numerical value in order to be used with many algorithms.  E.g. Linear or logistic regression.


## Question #7
Transform one of the categorical variables into numerical using label encoder method

In [14]:
df["Embarked"] = df["Embarked"].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    int64   
 2   Pclass       891 non-null    int64   
 3   Name         891 non-null    object  
 4   Sex          825 non-null    object  
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         819 non-null    float64 
 10  Cabin        204 non-null    object  
 11  Embarked     889 non-null    category
dtypes: category(1), float64(2), int64(5), object(4)
memory usage: 77.7+ KB


In [15]:
df["Embarked_code"] = df["Embarked"].cat.codes
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_code
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,2


## Question #8
Transform gender into numerical variable using dummy coding

In [16]:
# print(pd.get_dummies(data,columns=[‘Sex]).head())
pd.get_dummies(df, columns=['Sex'])

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Embarked_code,Sex_female,Sex_male
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0,1,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,2,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,S,2,0,1
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,S,2,1,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,NaN,S,2,1,0
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,C,0,0,1
